In [67]:
import yaml
from jinja2 import Template
f1=open("junos.j2")
jt=f1.read()
f1.close()
f1=open("topo1.yaml")
d1=yaml.load(f1,Loader=yaml.FullLoader)
f1.close()

In [41]:
d1['switch']['leaf2']

{'type': 'leaf',
 'lo0': '2001:dead:beef:ffff::12',
 'router_id': '10.1.100.12',
 'intf': ['xe-0/0/0.0', 'xe-0/0/1.0'],
 'bgp': {'underlay': {'local-as': 65012},
  'overlay': {'local-as': 4200065012,
   'neighbor': [{'remote-peer': '2001:dead:beef:ffff::1',
     'remote-as': 4200065001},
    {'remote-peer': '2001:dead:beef:ffff::2', 'remote-as': 4200065002}]}}}

In [69]:
d1['switch']['spine1']['as_list']=d1['global']['as_list']
print(Template(jt).render(d1['switch']['leaf1']))

interfaces {
	xe-0/0/0.0 {
			unit 0 {
				family inet;
				family inet6;
			}
		}
	xe-0/0/1.0 {
			unit 0 {
				family inet;
				family inet6;
			}
		}
	lo0 {
	unit 0 {
		family inet6 {
			address 2001:dead:beef:ffff::11;
		}
	}
	}
}
policy-options {
    policy-statement DIRECT-RTS {
		term 1 {
			from protocol direct;
			then accept;
		}
		term default {
			then reject;
		}
		}
	
    policy-statement lb {
        then {
            load-balance per-packet;
        }
    }
    as-list a-list members ;
}
routing-options {
    router-id 10.1.100.11;
    forwarding-table {
        export lb;
    }
}
protocols {
    router-advertisement {
		interface xe-0/0/0.0.0;
		interface xe-0/0/1.0.0;
		}
    bgp {
        group auto-disc {
            family inet6 {
                unicast;
            }
            export DIRECT-RTS;
            local-as 65011;
            multipath {
                multiple-as;
            }
            dynamic-neighbor FABRIC {
                peer-auto-discover